In [1]:
# Import the data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import os
import lightgbm as lgb
import xgboost as xgb# Encode the y data with the label encoder




data_cleaned = pd.read_csv('./clean-data/2017-fCC-New-Coders-Survey-Data.csv')
data_cleaned.head()

C:\Users\philk\AppData\Local\Temp\ipykernel_92592\2659837848.py:14: DtypeWarning: Columns (17,62) have mixed types. Specify dtype option on import or set low_memory=False.
  data_cleaned = pd.read_csv('./clean-data/2017-fCC-New-Coders-Survey-Data.csv')


,Age,AttendedBootcamp,BootcampFinish,BootcampLoanYesNo,BootcampName,BootcampRecommend,ChildrenNumber,CityPopulation,CodeEventConferences,CodeEventDjangoGirls,...,YouTubeFCC,YouTubeFunFunFunction,YouTubeGoogleDev,YouTubeLearnCode,YouTubeLevelUpTuts,YouTubeMIT,YouTubeMozillaHacks,YouTubeOther,YouTubeSimplilearn,YouTubeTheNewBoston
0,27.0,0.0,NaN,NaN,NaN,NaN,NaN,more than 1 million,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34.0,0.0,NaN,NaN,NaN,NaN,NaN,"less than 100,000",NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21.0,0.0,NaN,NaN,NaN,NaN,NaN,more than 1 million,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN
3,26.0,0.0,NaN,NaN,NaN,NaN,NaN,"between 100,000 and 1 million",NaN,NaN,...,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,20.0,0.0,NaN,NaN,NaN,NaN,NaN,"between 100,000 and 1 million",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
data_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18175 entries, 0 to 18174
Columns: 136 entries, Age to YouTubeTheNewBoston
dtypes: float64(105), object(31)
memory usage: 18.9+ MB


In [3]:
data_cleaned.dtypes


Age                    float64
AttendedBootcamp       float64
BootcampFinish         float64
BootcampLoanYesNo      float64
BootcampName            object
                        ...   
YouTubeMIT             float64
YouTubeMozillaHacks    float64
YouTubeOther            object
YouTubeSimplilearn     float64
YouTubeTheNewBoston    float64
Length: 136, dtype: object

In [4]:
# Check for missing values
missing_values = data_cleaned.isnull().sum()
print(missing_values)

Age                     2808
AttendedBootcamp         466
BootcampFinish         17106
BootcampLoanYesNo      17096
BootcampName           17226
                       ...  
YouTubeMIT             14848
YouTubeMozillaHacks    17553
YouTubeOther           17027
YouTubeSimplilearn     17974
YouTubeTheNewBoston    15215
Length: 136, dtype: int64


In [5]:
# Step 1: Handle missing values by removing rows with missing values
data_cleaned = data_cleaned.dropna()

In [6]:
# Step 2: Remove duplicate records
data_cleaned = data_cleaned.drop_duplicates()

In [7]:
# Step 3: Merge Columns
# Merge 'Do you financially support any dependents?' and 'Do you have children?' into 'Dependents'
def merge_dependents(row):
    # Check if both columns exist and apply logic
    if 'Do you financially support any dependents?' in row.index and 'Do you have children?' in row.index:
        if row['Do you financially support any dependents?'] == 'Yes' or row['Do you have children?'] == 'Yes':
            return 'Yes'
        else:
            return 'No'
    return np.nan  # Return NaN if columns do not exist
# Apply the function to create the 'Dependents' column
data_cleaned['Dependents'] = data_cleaned.apply(merge_dependents, axis=1)
# Merge 'Do you have student loan debt?', 'Do you have any debt?', and 'Do you have a home mortgage?' into 'Debt status category'
def merge_debt(row):
    if 'Do you have student loan debt?' in row.index and row['Do you have student loan debt?'] == 'Yes':
        return 'Student Loan'
    elif 'Do you have a home mortgage?' in row.index and row['Do you have a home mortgage?'] == 'Yes':
        return 'Mortgage'
    elif 'Do you have any debt?' in row.index and row['Do you have any debt?'] == 'Yes':
        return 'Other Debt'
    else:
        return 'No Debt'
# Apply the function to create the 'Debt status category' column
data_cleaned['Debt status category'] = data_cleaned.apply(merge_debt, axis=1)
# Merge 'Other' columns ('Other' gender, employment/school, career)
def merge_other(row):
    if 'Other' in row.index and pd.notna(row['Other']):
        return row['Other']
    elif 'Other.1' in row.index and pd.notna(row['Other.1']):
        return row['Other.1']
    elif 'Other.2' in row.index and pd.notna(row['Other.2']):
        return row['Other.2']
    return np.nan
# Apply the function to merge 'Other' columns
data_cleaned['Other'] = data_cleaned.apply(merge_other, axis=1)














In [8]:
# Step 4: Remove irrelevant columns
columns_to_remove = ['Submit Date (UTC)', 'Start Date (UTC)', 'Network ID', 'Other.1', 'Other.2', 'Other']
data_cleaned = data_cleaned.drop(columns=[col for col in columns_to_remove if col in data_cleaned.columns])
# Step 5: Handle any inconsistencies in the data (assuming none for now)
# The data is now cleaned and ready for machine learning algorithms.
# Optional: Save the cleaned data to a new CSV file
data_cleaned.to_csv('cleaned_survey_data.csv', index=False)

In [9]:
# Step 6: Remove irrelevant columns if they exist in the dataset
columns_to_remove = ['Submit Date (UTC)', 'Start Date (UTC)', 'Network ID', '#', 'Other', 'Other.1', 'Other.2']
# Check if each column exists before dropping
data_cleaned = data_cleaned.drop(columns=[col for col in columns_to_remove if col in data_cleaned.columns], errors='ignore')

In [10]:
# Step 7: Display cleaned data (or save it to a new CSV file)
print(data_cleaned.head())  # Display the first few rows
# If you want to save the cleaned data to a new file
data_cleaned.to_csv('cleaned_data.csv', index=False)

Empty DataFrame
Columns: [Age, AttendedBootcamp, BootcampFinish, BootcampLoanYesNo, BootcampName, BootcampRecommend, ChildrenNumber, CityPopulation, CodeEventConferences, CodeEventDjangoGirls, CodeEventFCC, CodeEventGameJam, CodeEventGirlDev, CodeEventHackathons, CodeEventMeetup, CodeEventNodeSchool, CodeEventNone, CodeEventOther, CodeEventRailsBridge, CodeEventRailsGirls, CodeEventStartUpWknd, CodeEventWkdBootcamps, CodeEventWomenCode, CodeEventWorkshops, CommuteTime, CountryCitizen, CountryLive, EmploymentField, EmploymentFieldOther, EmploymentStatus, EmploymentStatusOther, ExpectedEarning, FinanciallySupporting, FirstDevJob, Gender, GenderOther, HasChildren, HasDebt, HasFinancialDependents, HasHighSpdInternet, HasHomeMortgage, HasServedInMilitary, HasStudentDebt, HomeMortgageOwe, HoursLearning, ID.x, ID.y, Income, IsEthnicMinority, IsReceiveDisabilitiesBenefits, IsSoftwareDev, IsUnderEmployed, JobApplyWhen, JobInterestBackEnd, JobInterestDataEngr, JobInterestDataSci, JobInterestDevO

In [11]:
# Step 1: Label Encoding for small categories and ordinal features
label_encoder = LabelEncoder()
# Label encode Gender (if binary or small categories)
if 'Gender' in data_cleaned.columns:
    data_cleaned['Gender_Encoded'] = label_encoder.fit_transform(data_cleaned['Gender'])
# Label encode Degree Level (ordinal)
if 'Degree Level' in data_cleaned.columns:
    data_cleaned['Degree_Level_Encoded'] = label_encoder.fit_transform(data_cleaned['Degree Level'])
# Label encode Employment Status
if 'Employment Status' in data_cleaned.columns:
    data_cleaned['Employment_Status_Encoded'] = label_encoder.fit_transform(data_cleaned['Employment Status'])
# Label encode Student Loan Debt (Yes/No or binary)
if 'Do you have student loan debt?' in data_cleaned.columns:
    data_cleaned['Student_Loan_Debt_Encoded'] = label_encoder.fit_transform(data_cleaned['Do you have student loan debt?'])
# Label encode Family Responsibilities (Yes/No for dependents, children)
if 'Dependents' in data_cleaned.columns:
    data_cleaned['Dependents_Encoded'] = label_encoder.fit_transform(data_cleaned['Dependents'])
# Step 2: One-Hot Encoding for unordered categorical variables
# One-hot encode Country of Residence
if 'Country of Residence' in data_cleaned.columns:
    data_cleaned = pd.get_dummies(data_cleaned, columns=['Country of Residence'], prefix='Country')
# One-hot encode Citizenship
if 'Citizenship' in data_cleaned.columns:
    data_cleaned = pd.get_dummies(data_cleaned, columns=['Citizenship'], prefix='Citizenship')
# One-hot encode Job Roles
if 'Job Roles' in data_cleaned.columns:
    data_cleaned = pd.get_dummies(data_cleaned, columns=['Job Roles'], prefix='Job_Role')
# One-hot encode Field of Study
if 'Field of Study' in data_cleaned.columns:
    data_cleaned = pd.get_dummies(data_cleaned, columns=['Field of Study'], prefix='Field_Study')
# One-hot encode Learning Preferences
if 'Learning Preferences' in data_cleaned.columns:
    data_cleaned = pd.get_dummies(data_cleaned, columns=['Learning Preferences'], prefix='Learning_Pref')
# One-hot encode Coding Events/Workshops
if 'Coding Events/Workshops' in data_cleaned.columns:
    data_cleaned = pd.get_dummies(data_cleaned, columns=['Coding Events/Workshops'], prefix='Coding_Event')
# One-hot encode Employment Type Preferences
if 'Employment Type Preferences' in data_cleaned.columns:
    data_cleaned = pd.get_dummies(data_cleaned, columns=['Employment Type Preferences'], prefix='Employment_Type')
# Optional: Step 3: Handling High-Cardinality Features
# For very high-cardinality categorical features like 'Country', if dimensionality becomes an issue,
# consider using binary encoding from the 'category_encoders' library.
# Example: Using binary encoding for 'Country' feature with many categories
# Install the category_encoders package if needed:
# !pip install category_encoders
# from category_encoders import BinaryEncoder
# encoder = BinaryEncoder(cols=['Country'])
# data_cleaned = encoder.fit_transform(data_cleaned)
# Save the encoded dataset to a new CSV file
data_cleaned.to_csv('encoded_survey_data.csv', index=False)


In [12]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Load the dataset (replace with your actual file path)
data_cleaned = pd.read_csv('./clean-data/2017-fCC-New-Coders-Survey-Data.csv')
# Step 1: Identify Numerical Columns
# Select only numerical columns (int64 and float64)
numerical_columns = data_cleaned.select_dtypes(include=['int64', 'float64']).columns.tolist()
# Step 2: Exclude binary or categorical columns from scaling
# Define the binary or categorical columns that should not be scaled
exclude_columns = ['Dependents', 'Gender', 'Employment Status']  # Add any other binary or categorical columns here
# Identify the numerical columns to scale by excluding the binary/categorical columns
columns_to_scale = [col for col in numerical_columns if col not in exclude_columns]
print("Columns to scale: ", columns_to_scale)
# Step 3: Normalize the selected columns using MinMaxScaler
scaler = MinMaxScaler()
data_cleaned[columns_to_scale] = scaler.fit_transform(data_cleaned[columns_to_scale])
# Step 4: Standardize the selected columns using StandardScaler (if you want to standardize instead of normalize)
# scaler = StandardScaler()
# data_cleaned[columns_to_scale] = scaler.fit_transform(data_cleaned[columns_to_scale])
# The data is now normalized (or standardized if you uncomment the StandardScaler)
# Optional: Save the final processed data
data_cleaned.to_csv('final_processed_data.csv', index=False)
# Check the first few rows of the scaled data
print(data_cleaned.head())

C:\Users\philk\AppData\Local\Temp\ipykernel_92592\3691866491.py:3: DtypeWarning: Columns (17,62) have mixed types. Specify dtype option on import or set low_memory=False.
  data_cleaned = pd.read_csv('./clean-data/2017-fCC-New-Coders-Survey-Data.csv')


Columns to scale:  ['Age', 'AttendedBootcamp', 'BootcampFinish', 'BootcampLoanYesNo', 'BootcampRecommend', 'ChildrenNumber', 'CodeEventConferences', 'CodeEventDjangoGirls', 'CodeEventFCC', 'CodeEventGameJam', 'CodeEventGirlDev', 'CodeEventHackathons', 'CodeEventMeetup', 'CodeEventNodeSchool', 'CodeEventNone', 'CodeEventRailsBridge', 'CodeEventRailsGirls', 'CodeEventStartUpWknd', 'CodeEventWkdBootcamps', 'CodeEventWomenCode', 'CodeEventWorkshops', 'ExpectedEarning', 'FinanciallySupporting', 'FirstDevJob', 'HasChildren', 'HasDebt', 'HasFinancialDependents', 'HasHighSpdInternet', 'HasHomeMortgage', 'HasServedInMilitary', 'HasStudentDebt', 'HomeMortgageOwe', 'HoursLearning', 'Income', 'IsEthnicMinority', 'IsReceiveDisabilitiesBenefits', 'IsSoftwareDev', 'IsUnderEmployed', 'JobInterestBackEnd', 'JobInterestDataEngr', 'JobInterestDataSci', 'JobInterestDevOps', 'JobInterestFrontEnd', 'JobInterestFullStack', 'JobInterestGameDev', 'JobInterestInfoSec', 'JobInterestMobile', 'JobInterestProjMngr'

In [13]:
# Check for missing values in 'AttendedBootcamp' and 'BootcampFinish'
print(data_cleaned['AttendedBootcamp'].isnull().sum())
print(data_cleaned['BootcampFinish'].isnull().sum())

466
17106


In [14]:
# Drop rows where 'AttendedBootcamp' or 'BootcampFinish' have NaN values
data_cleaned = data_cleaned.dropna(subset=['AttendedBootcamp', 'BootcampFinish'])

In [15]:
# Define features (X) and targets ('AttendedBootcamp' and 'BootcampFinish') again after removing NaNs
X = data_cleaned.drop(columns=['AttendedBootcamp', 'BootcampFinish'])  # Features
y_attend = data_cleaned['AttendedBootcamp']  # Target 1: Attend Bootcamp
y_finish = data_cleaned['BootcampFinish']  # Target 2: Finish Bootcamp

In [16]:
from sklearn.model_selection import train_test_split
# Split data for 'AttendedBootcamp' prediction with stratified sampling
X_train_attend, X_test_attend, y_train_attend, y_test_attend = train_test_split(X, y_attend, test_size=0.3, stratify=y_attend, random_state=42)

In [17]:
# Split data for 'BootcampFinish' prediction with stratified sampling
X_train_finish, X_test_finish, y_train_finish, y_test_finish = train_test_split(X, y_finish, test_size=0.3, stratify=y_finish, random_state=42)

In [18]:
# Define file paths for each member (you can modify these paths as needed)
path_ayana = "datasets/ayana/"
path_roberta = "datasets/roberta/"
path_dom = "datasets/dom/"
path_phillip = "datasets/phillip/"


In [19]:
# Create directories if they don't exist
os.makedirs(path_ayana, exist_ok=True)
os.makedirs(path_roberta, exist_ok=True)
os.makedirs(path_dom, exist_ok=True)
os.makedirs(path_phillip, exist_ok=True)

In [20]:
# # Ensure the paths exist (if necessary)
# # For Ayana: Logistic Regression & Decision Trees (AttendedBootcamp and BootcampFinish)
# # Export Ayana's training and test datasets for AttendedBootcamp
# X_train_attend.to_csv(path_ayana + "X_train_attend.csv", index=False)
# y_train_attend.to_csv(path_ayana + "y_train_attend.csv", index=False)
# X_test_attend.to_csv(path_ayana + "X_test_attend.csv", index=False)
# y_test_attend.to_csv(path_ayana + "y_test_attend.csv", index=False)

In [21]:
# # Export Ayana's training and test datasets for BootcampFinish
# X_train_finish.to_csv(path_ayana + "X_train_finish.csv", index=False)
# y_train_finish.to_csv(path_ayana + "y_train_finish.csv", index=False)
# X_test_finish.to_csv(path_ayana + "X_test_finish.csv", index=False)
# y_test_finish.to_csv(path_ayana + "y_test_finish.csv", index=False)

In [22]:
# For Roberta: Random Forest
X_train_attend.to_csv(path_roberta + "X_train_attend.csv", index=False)
y_train_attend.to_csv(path_roberta + "y_train_attend.csv", index=False)
X_test_attend.to_csv(path_roberta + "X_test_attend.csv", index=False)
y_test_attend.to_csv(path_roberta + "y_test_attend.csv", index=False)
X_train_finish.to_csv(path_roberta + "X_train_finish.csv", index=False)
y_train_finish.to_csv(path_roberta + "y_train_finish.csv", index=False)
X_test_finish.to_csv(path_roberta + "X_test_finish.csv", index=False)
y_test_finish.to_csv(path_roberta + "y_test_finish.csv", index=False)

In [23]:
# For Dom: SVM & KNN
#X_train_attend.to_csv(path_dom + "X_train_attend.csv", index=False)
#y_train_attend.to_csv(path_dom + "y_train_attend.csv", index=False)
# X_test_attend.to_csv(path_dom + "X_test_attend.csv", index=False)
# y_test_attend.to_csv(path_dom + "y_test_attend.csv", index=False)
# X_train_finish.to_csv(path_dom + "X_train_finish.csv", index=False)
# y_train_finish.to_csv(path_dom + "y_train_finish.csv", index=False)
# X_test_finish.to_csv(path_dom + "X_test_finish.csv", index=False)
# y_test_finish.to_csv(path_dom + "y_test_finish.csv", index=False)


In [24]:
# you can export the clean dataset for each member as well
data_cleaned.to_csv(path_ayana + "clean_dataset.csv", index=False)
data_cleaned.to_csv(path_roberta + "clean_dataset.csv", index=False)
data_cleaned.to_csv(path_dom + "clean_dataset.csv", index=False)
data_cleaned.to_csv(path_phillip + "clean_dataset.csv", index=False)



# RandomForest DATA Analysis

In [26]:
##Import
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

In [27]:
X_test_finish

,Age,BootcampLoanYesNo,BootcampName,BootcampRecommend,ChildrenNumber,CityPopulation,CodeEventConferences,CodeEventDjangoGirls,CodeEventFCC,CodeEventGameJam,...,YouTubeFCC,YouTubeFunFunFunction,YouTubeGoogleDev,YouTubeLearnCode,YouTubeLevelUpTuts,YouTubeMIT,YouTubeMozillaHacks,YouTubeOther,YouTubeSimplilearn,YouTubeTheNewBoston
15799,0.288889,0.0,NaN,1.0,NaN,more than 1 million,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7643,0.266667,0.0,General Assembly,1.0,NaN,more than 1 million,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11330,NaN,1.0,We Can Code IT,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
14366,0.288889,1.0,Science to Data Science,1.0,NaN,"less than 100,000",NaN,NaN,0.0,NaN,...,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
12291,NaN,1.0,Hack Reactor,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9358,0.300000,0.0,codeU,0.0,NaN,more than 1 million,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
804,0.333333,0.0,Logit Academy,0.0,NaN,"between 100,000 and 1 million",NaN,NaN,0.0,NaN,...,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
10408,NaN,0.0,Academic Work Academy,1.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1873,0.533333,0.0,Galvanize,1.0,NaN,"between 100,000 and 1 million",NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# Initialize the Decision Tree model for finish data
dt_model_finish = DecisionTreeClassifier(random_state=42)

In [51]:
# Convert categorical variables to dummy/indicator variables
# X_train_finish = pd.get_dummies(X_train_finish) -------------------------------- chnage back if needed 10/29
# X_test_finish = pd.get_dummies(X_test_finish)   -------------------------------- chnage back if needed 10/29

# Align the columns of train and test sets
X_train_finish, X_test_finish = X_train_finish.align(X_test_finish, join='left', axis=1, fill_value=0)

In [55]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

# Example data (replace with your actual data)
X_train_finish = pd.DataFrame({
    'feature1': [1, 2, 3],
    'category': ['Coder Camps', 'Coder Camps', 'Other']
})
y_train_finish = [0, 1, 0]

# Create a column transformer to apply one-hot encoding to the categorical column
column_transformer = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['category'])  # Apply OneHotEncoder to 'category' column
    ],
    remainder='passthrough'  # Keep the other columns as they are
)

# Create a pipeline that first transforms the data and then fits the model
dt_model_finish = Pipeline(steps=[
    ('preprocessor', column_transformer),  # Preprocessing step
    ('classifier', DecisionTreeClassifier())  # Model fitting step
])

# Fit the model
dt_model_finish.fit(X_train_finish, y_train_finish)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat', OneHotEncoder(),
                                                  ['category'])])),
                ('classifier', DecisionTreeClassifier())])

In [63]:
# Check if required columns are present in X_test_finish
required_columns = ['category', 'feature1']  # List of required columns

# Ensure that all required columns are in the test DataFrame
missing_columns = [col for col in required_columns if col not in X_test_finish.columns]

if missing_columns:
    # Handle the missing columns case
    raise ValueError(f"Missing columns in test data: {missing_columns}")

ValueError: Missing columns in test data: ['category', 'feature1']

In [65]:


# Make predictions
y_pred_finish = dt_model_finish.predict(X_test_finish)

# Evaluate the model
print("Accuracy (Finish):", accuracy_score(y_test_finish, y_pred_finish))
print(classification_report(y_test_finish, y_pred_finish))

ValueError: columns are missing: {'category', 'feature1'}

In [67]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# Create the grid search object
grid_search_finish = GridSearchCV(estimator=dt_model_finish, param_grid=param_grid, 
                                   scoring='accuracy', cv=5, n_jobs=-1, verbose=2)

# Fit the grid search model
grid_search_finish.fit(X_train_finish, y_train_finish)

# Best parameters for attend data
print("Best Parameters (finish):", grid_search_finish.best_params_)

# Evaluate the best model
best_dt_model_finish = grid_search_finish.best_estimator_
y_pred_finish_best = dt_model_finish.predict(X_test_finish)

# Print evaluation metrics
print("Accuracy (Finish Best):", accuracy_score(y_test_finish, y_pred_finish_best))
print(classification_report(y_test_finish, y_pred_finish_best))

Fitting 5 folds for each of 36 candidates, totalling 180 fits


ValueError: Cannot have number of splits n_splits=5 greater than the number of samples: n_samples=3.

In [69]:
### Load the Data 
path_roberta = "datasets/roberta/"

In [71]:
# For Roberta: Random Forest
# X_train_attend.to_csv(path_roberta + "X_train_attend.csv", index=False)
# y_train_attend.to_csv(path_roberta + "y_train_attend.csv", index=False)
# X_test_attend.to_csv(path_roberta + "X_test_attend.csv", index=False)
# y_test_attend.to_csv(path_roberta + "y_test_attend.csv", index=False)
X_train_finish.to_csv(path_roberta + "X_train_finish.csv", index=False)
y_train_finish_df = pd.DataFrame(y_train_finish)
y_train_finish_df.to_csv(path_roberta + "y_train_finish.csv", index=False)
X_test_finish.to_csv(path_roberta + "X_test_finish.csv", index=False)
y_test_finish_df = pd.DataFrame(y_test_finish)
y_test_finish_df.to_csv(path_roberta + "y_test_finish.csv", index=False)

In [73]:
# ###Train a Random Forest Model
# X_train_finish = pd.read_csv("datasets/roberta/X_train_finish.csv")
# y_train_finish = pd.read_csv("datasets/roberta/y_train_finish.csv").values.ravel()  
# X_test_finish = pd.read_csv("datasets/roberta/X_test_finish.csv")
# y_test_finish = pd.read_csv("datasets/roberta/y_test_finish.csv").values.ravel()

# Convert categorical variables to dummy/indicator variables
# X_train_finish = pd.get_dummies(X_train_finish)
# X_test_finish = pd.get_dummies(X_test_finish)

# Align the columns of train and test sets
X_train_finish, X_test_finish = X_train_finish.align(X_test_finish, join='left', axis=1, fill_value=0)

rf_model_finish = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_finish.fit(X_train_finish, y_train_finish)
y_pred_finish = rf_model_finish.predict(X_test_finish)

print("Accuracy (Finish):", accuracy_score(y_test_finish, y_pred_finish))
print(classification_report(y_test_finish, y_pred_finish))

NameError: name 'RandomForestClassifier' is not defined

In [75]:
importance = rf_model_finish.feature_importances_
feature_df = pd.DataFrame({"Feature": X.columns,'Gini Importance': importance}).sort_values('Gini Importance', ascending = False)
feature_df

NameError: name 'rf_model_finish' is not defined

In [77]:
import matplotlib.pyplot as plt

feature_10 = feature_df[:8]

plt.barh(feature_10['Feature'], feature_10['Gini Importance'], color='blue', )
plt.xlabel('Gini Importance')
plt.ylabel('Features')
plt.title('Feature Imp - Bootcamp Graduation')
plt.show()

NameError: name 'feature_df' is not defined

In [79]:
# you can export the clean dataset for each member as well
data_cleaned.to_csv(path_ayana + "clean_dataset.csv", index=False)
data_cleaned.to_csv(path_roberta + "clean_dataset.csv", index=False)
data_cleaned.to_csv(path_dom + "clean_dataset.csv", index=False)
data_cleaned.to_csv(path_phillip + "clean_dataset.csv", index=False)

In [81]:
# Find the percentage of null values in each column
X_train_attend.isna().sum()/len(X_train_attend)
X_train_finish.isna().sum()/len(X_train_finish)
#X_train.isna().sum()/len(X_train)

feature1    0.0
category    0.0
dtype: float64

In [83]:
# Explore each column with missing values to determine the best fill strategy
# First the job column
X_train['<Choose Column>'].value_counts()

NameError: name 'X_train' is not defined

# XGBoost DATA Analysis

In [86]:
# For Phillip: XGBoost & LightGBM
X_train_attend.to_csv(path_phillip + "X_train_attend.csv", index=False)
y_train_attend.to_csv(path_phillip + "y_train_attend.csv", index=False)
X_test_attend.to_csv(path_phillip + "X_test_attend.csv", index=False)
y_test_attend.to_csv(path_phillip + "y_test_attend.csv", index=False)
X_train_finish.to_csv(path_phillip + "X_train_finish.csv", index=False)
y_train_finish.to_csv(path_phillip + "y_train_finish.csv", index=False)
X_test_finish.to_csv(path_phillip + "X_test_finish.csv", index=False)
y_test_finish.to_csv(path_phillip + "y_test_finish.csv", index=False)

AttributeError: 'list' object has no attribute 'to_csv'

In [ ]:
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder  # Import LabelEncoder

# Create a new LabelEncoder specifically for the target variable
target_le = LabelEncoder()

# Fit the target encoder on the training labels
y_train_encoded = target_le.fit_transform(y_train_finish)

In [ ]:
# Convert categorical columns in X_train_finish and X_test_finish to numerical values
# Assuming X_train_finish and X_test_finish are pandas DataFrames
for column in X_train_finish.select_dtypes(include=['object']).columns:
    le = LabelEncoder()  # Create a new LabelEncoder for each column

    # Fit on the combined unique values from both train and test data
    all_values = pd.concat([X_train_finish[column], X_test_finish[column]]).unique()
    le.fit(all_values)

    X_train_finish[column] = le.transform(X_train_finish[column])  # Encode the training data
    X_test_finish[column] = le.transform(X_test_finish[column])  # Encode the test data# Convert categorical columns in X_train_finish and X_test_finish to numerical values


In [ ]:
X_test_finish[['BootcampName']].head()

In [ ]:
# Use the target encoder to transform the test labels, handling unseen labels
try:
    y_test_encoded = target_le.transform(y_test_finish)  # This will raise an error if unseen labels exist
except ValueError as e:
    print(f"Warning: {e}. Encoding unseen labels as -1.")
    # Get unseen labels and add them to the encoder
    unseen_labels = set(y_test_finish) - set(target_le.classes_)
    for label in unseen_labels:
        target_le.classes_ = np.append(target_le.classes_, label)
    y_test_encoded = target_le.transform(y_test_finish)

In [ ]:

# Creating an XGBoost classifier
model = xgb.XGBClassifier()

In [ ]:
# Training the model on the training data
model.fit(X_train_finish, y_train_encoded)

In [ ]:
X_train_finish[['BootcampName']].head()

In [ ]:
X_test_finish[['BootcampName']].head()

In [ ]:
# Making predictions on the test set
predictions = model.predict(X_test_finish)

In [ ]:

# Evaluate the model
print(f'Training Score: {model.score(X_train_finish, y_train_encoded)}')
print(f'Testing Score: {model.score(X_test_finish, y_test_encoded)}')

In [ ]:
# Create a loop to vary the max_depth parameter
# Make sure to record the train and test scores 
# for each pass.

# Depths should span from 1 up to 40 in steps of 2
depths = range(1, 20, 2)

# The scores dataframe will hold depths and scores
# to make plotting easy
scores = {'train': [], 'test': [], 'depth': []}

# Loop through each depth (this will take time to run)
for depth in depths:
    clf = xgb.XGBClassifier(max_depth=depth)
    clf.fit(X_train_finish, y_train_finish)

    train_score = clf.score(X_train_finish, y_train_finish)
    test_score = clf.score(X_test_finish, y_test_finish)

    scores['depth'].append(depth)
    scores['train'].append(train_score)
    scores['test'].append(test_score)

# Create a dataframe from the scores dictionary and
# set the index to depth
scores_df = pd.DataFrame(scores).set_index('depth')

scores_df.head()

In [ ]:
# Plot the scores dataframe with the plot method
scores_df.plot()